# Práctica 4
### Lucía Cordero y Jorge Garcelán

## Contornos

In [ ]:
import cv2
import numpy as np

# Cargar la imagen
image = cv2.imread("./Imagenes/Imagenes/Piezas.png")

if image is None:
    print("Error!!")
    exit(1)

cv2.imshow("Input Image", image)

# Convertir a escala de grises y umbralizar
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image_thresh = cv2.threshold(image_gray, 128, 255, cv2.THRESH_BINARY_INV)

# Encontrar contornos
contours, _ = cv2.findContours(image_thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Dibujar contornos
contour_image = np.zeros_like(image)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
for idx, contour in enumerate(contours):
    cv2.drawContours(contour_image, contours, idx, colors[idx % 3], 1)

cv2.imshow("Contours", contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Descriptores

In [ ]:
import cv2
import numpy as np
import csv

# Cargar la imagen
image = cv2.imread("./Imagenes/Imagenes/pieza3.png")
nombre_archivo = "descriptores.txt"

if image is None:
    print("Error!!")
    exit(1)

cv2.imshow("Input Image", image)

# Convertir a escala de grises y umbralizar
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image_thresh = cv2.threshold(image_gray, 128, 255, cv2.THRESH_BINARY_INV)

# Encontrar contornos
contours, _ = cv2.findContours(image_thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

print(f"El número de piezas es {len(contours)}")

perimetro = []
area = []
momentos = []
hu_moments = []

# Extraer descriptores
for contour in contours:
    perimetro.append(cv2.arcLength(contour, True))
    area.append(cv2.contourArea(contour))
    momentos.append(cv2.moments(contour))
    hu = cv2.HuMoments(momentos[-1]).flatten()
    hu_moments.append(hu)

# Guardar descriptores en archivo CSV
with open(nombre_archivo, mode='w', newline='') as archivo:
    writer = csv.writer(archivo)
    writer.writerow(["perimetro", "area", "Hu1", "Hu2", "Hu3", "Hu4", "Hu5", "Hu6", "Hu7"])
    for i in range(len(contours)):
        writer.writerow([perimetro[i], area[i], *hu_moments[i]])

cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Bayes

In [3]:
print(f"train_data shape: {train_data.shape}")
print(f"response_data shape: {response_data.shape}")


train_data shape: (124, 9)
response_data shape: (124, 1)


In [16]:
import cv2
import numpy as np
import pickle

# Definir constantes
NUM_CLASES = 3
HU_MOMENTS = 3  # Cambiado a 3 para evitar error de dimensión
NUMDESCRIPTORES = HU_MOMENTS + 2  # Hu moments + perimetro + area
print(f"El número de descriptores es: {NUMDESCRIPTORES} con {HU_MOMENTS} momentos de Hu, perimetro y área")
CLASES = ["./Imagenes/Imagenes/Pieza1.png", "./Imagenes/Imagenes/Pieza2.png", "./Imagenes/Imagenes/Pieza3.png"]
NOMBRE_CLASES = ["Palillo", "Forma T", "Cruzada"]
IMAGEN_PIEZAS = "./Imagenes/Imagenes/Piezas.png"

def prepare_train_data(data, responses, ntrain_samples):
    sample_idx = np.zeros((1, data.shape[0]), dtype=np.uint8)
    sample_idx[0, :ntrain_samples] = 1
    var_type = np.ones((data.shape[1] + 1, 1), dtype=np.uint8) * cv2.ml.VAR_ORDERED
    var_type[-1, 0] = cv2.ml.VAR_CATEGORICAL
    return cv2.ml.TrainData_create(data, cv2.ml.ROW_SAMPLE, responses, sampleIdx=sample_idx, varType=var_type)

# Cargar imágenes de entrenamiento
imagenes_clase = [cv2.imread(img) for img in CLASES]

if any(img is None for img in imagenes_clase):
    print("Error al leer las imágenes de entrenamiento!!")
    exit(1)

contours_clase = []
for img in imagenes_clase:
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img_thresh = cv2.threshold(img_gray, 128, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(img_thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours_clase.append(contours)

nentre = sum(len(c) for c in contours_clase)
print(f"El número total de piezas utilizadas para entrenamiento es: {nentre}")

# Crear matrices para almacenar información
train_data = np.zeros((nentre, NUMDESCRIPTORES), dtype=np.float32)
response_data = np.zeros((nentre, 1), dtype=np.float32)

k = 0
for clase, contours in enumerate(contours_clase):
    for contour in contours:
        momentos = cv2.moments(contour)
        hu = cv2.HuMoments(momentos).flatten()[:HU_MOMENTS]  # Tomar solo los primeros 3 momentos
        perimetro = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        train_data[k, :HU_MOMENTS] = hu
        train_data[k, HU_MOMENTS] = perimetro
        train_data[k, HU_MOMENTS + 1] = area
        response_data[k, 0] = clase
        k += 1

# Preparar datos de entrenamiento
tdata = prepare_train_data(train_data, response_data, nentre)

# Entrenar clasificador de Bayes
bayes = cv2.ml.NormalBayesClassifier_create()
bayes.train(tdata)

# Guardar modelo con OpenCV
bayes.save("bayes_model.xml")

# Clasificación
clasificar_img = cv2.imread(IMAGEN_PIEZAS)
if clasificar_img is None:
    print("Error al leer la imagen para clasificar!!")
    exit(1)

img_gray = cv2.cvtColor(clasificar_img, cv2.COLOR_BGR2GRAY)
_, imagen_bin = cv2.threshold(img_gray, 128, 255, cv2.THRESH_BINARY_INV)
contorno_piezas, _ = cv2.findContours(imagen_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
print(f"El número total de piezas a clasificar es: {len(contorno_piezas)}")

imagen_color = cv2.cvtColor(imagen_bin, cv2.COLOR_GRAY2BGR)
for idx, contorno in enumerate(contorno_piezas):
    momentos = cv2.moments(contorno)
    hu = cv2.HuMoments(momentos).flatten()[:HU_MOMENTS]  # Tomar solo los primeros 3 momentos
    perimetro = cv2.arcLength(contorno, True)
    area = cv2.contourArea(contorno)
    test_data = np.array([np.concatenate((hu, [perimetro, area]))], dtype=np.float32)
    clase = int(bayes.predict(test_data)[1][0][0])
    print(f"La predicción es {clase} ({NOMBRE_CLASES[clase]})")
    cv2.drawContours(imagen_color, [contorno], -1, (0, 0, 255), 2)
    cv2.imshow("Clasificación", imagen_color)
    cv2.waitKey(0)

cv2.destroyAllWindows()

El número de descriptores es: 5 con 3 momentos de Hu, perimetro y área
El número total de piezas utilizadas para entrenamiento es: 124
El número total de piezas a clasificar es: 12
La predicción es 1 (Forma T)
La predicción es 1 (Forma T)
La predicción es 2 (Cruzada)
La predicción es 0 (Palillo)
La predicción es 1 (Forma T)
La predicción es 0 (Palillo)
La predicción es 0 (Palillo)
La predicción es 1 (Forma T)
La predicción es 2 (Cruzada)
La predicción es 2 (Cruzada)
La predicción es 0 (Palillo)
La predicción es 2 (Cruzada)
